In [1]:
#We get all dependencis we need for a Random Forest
#If you need a module do: "!pip install [module_name]""
import numpy as np
import pandas as pd
import secrets
import matplotlib.pylab as plt
import itertools
import pydot
import pickle
import sys

from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine
from collections import Counter
from subprocess import call
from IPython.display import Image

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import export_graphviz

In [2]:
# Get the URL of the video we want to create highlights on
url = ""

In [ ]:
# Get the data extracted from our video
db_conn_str = "mysql+pymysql://{:}:{:}@{:}/{:}".format(secrets.user, secrets.password, secrets.host, secrets.db)
df = pd.read_sql('SELECT * FROM clips WHERE url ="'+ url +'"', con=db_conn)

print('Number of Rows:', len(df.index))
db_conn.dispose()

In [ ]:
features_df = df[['pitch', 'amplitude', 'subjectivity', 'polarity', 'pred_excitement', 'isHighlight']]

In [ ]:
# Separate our tags and features
y = np.array(features_df.pop('isHighlight'))
X = features_df
X.head()

In [ ]:
# Get model predictions
rf_predictions = df['pred_highlight_rf']
nn_predictions = df['pred_highlight_nn']

In [ ]:
# get youtube embeded url
def formatYoutubeUrl(url, start, end):
  embed_url = url.replace("watch?v=", "embed/")
  params = "?start={:}&end={:}&autoplay=1".format(start, end)
  return embed_url + params

def printUrls(df):
    for idx, row in df.iterrows():
        print(formatYoutubeUrl(row['url'], row['start'], row['end']))

In [ ]:
# all non-highlight clips
non_highlight_rf = df[df['pred_highlight_rf'] == 0]
non_highlight_nn = df[df['pred_highlight_nn'] == 0]

# all non-highlight clips
highlight_rf = df[df['pred_highlight_rf'] == 1]
highlight_nn = df[df['pred_highlight_rf'] == 1]

# print URLs
print('=== NON-HIGHLIGHT RF ===')
printUrls(non_highlight_rf)

print('=== HIGHLIGHT RF ===')
printUrls(highlight_rf)

print('=== NON-HIGHLIGHT NN ===')
printUrls(non_highlight_nn)

print('=== HIGHLIGHT NN ===')
printUrls(highlight_nn)

In [ ]:
# Output Predictions vs Actual results
for idx, row in dataset_by_url.iterrows():
    print(formatYoutubeUrl(row['url'], row['start'], row['end']), "Predicted RF: " ,row['pred_highlight_rf'], "Predicted NN: " ,row['pred_highlight_nn'])

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import time

def secondsToTime(secs):
    m, s = divmod(secs, 60)
    return "{:}:{:}".format(m, s)

def realTimeHypeGraph(dataset):
    fig = plt.figure(figsize=(8,4))
    ax = fig.add_subplot(111)
    plt.ion()

    fig.show()
    fig.canvas.draw()

    x = []
    y1 = []
    y2 = []

    for idx, row in dataset.iterrows():
        ax.clear()
        ax.set_ylim([-0.5, 2.5])
        
        videoTime = row['start'] / 60
        x.append(videoTime)
        y1.append(row['y_predict_rf'])
        y2.append(row['y_predict_nn'] + 0.02)

        ax.plot(x[-20:], y1[-20:], label='random_forest', alpha=0.7)
        ax.plot(x[-20:], y2[-20:], label='neural_network', alpha=0.7)
        
        plt.xlabel('Time (s)')
        plt.ylabel('Highlight')
        plt.legend()
        
        fig.canvas.draw()
        time.sleep(2)

In [ ]:
from IPython.display import HTML

youtubeEmbedUrl = url.replace('watch?v=', 'embed/')
# Youtube
HTML("<iframe width='560' height='315' src='{:}'</iframe>".format(youtubeEmbedUrl))

In [ ]:
# run this cell right before you click play on the youtube video
dataset = dataset_by_url[dataset_by_url['start'] >= 1140] # you can customize the start time here
realTimeHypeGraph(dataset)

In [ ]:
# Display a community-made highlight reel based on video
community_url =""

youtubeEmbedUrl = community_url.replace('watch?v=', 'embed/')
# Youtube
HTML("<iframe width='560' height='315' src='{:}'</iframe>".format(youtubeEmbedUrl))

In [ ]:
# Display our highlight reel based on video
from IPython.display import Video

embedVideo = Video("datastore/videos/test.mp4")

HTML("<iframe width='560' height='315' src='{:}'</iframe>".format(ymbedUrl))